# Sub Question Query Engine - LlamaIndex + KDB.AI

Note: This example requires a KDB.AI endpoint and API key. Sign up for a free [KDB.AI account](https://kdb.ai/offerings/).

In this notebook, we will walk through using KDB.AI and LlamaIndex's Sub Question Query Engine to create a RAG pipeline on a state of the union speech. The goal of the Sub Question Query Engine is to easily handle user queries that contain multiple questions within them by splitting the questions up.

## Install requirements

**Install required packages**

In [1]:
%pip install llama-index llama-index-embeddings-huggingface llama-index-llms-openai llama-index-readers-file llama-index-vector-stores-kdbai
%pip install kdbai_client sentence-transformers

**Helper Library - To allow nested loop events**

---



In [2]:
import nest_asyncio

nest_asyncio.apply()

## Downloading data

**Libraries**

In [3]:
import os
import urllib.request

**Data directories and paths**

In [4]:
# Root path
root_path = os.path.abspath(os.getcwd())

# Data directory and path
data_dir = "data"
data_path = os.path.join(root_path, data_dir)
if not os.path.exists(data_path):
    os.mkdir(data_path)

**Downloading text**

In [5]:
text_url = "https://raw.githubusercontent.com/KxSystems/kdbai-samples/main/retrieval_augmented_generation/data/state_of_the_union.txt"
with urllib.request.urlopen(text_url) as response:
    text_content = response.read().decode("utf-8")

text_file_name = text_url.split('/')[-1]
text_path = os.path.join(data_path, text_file_name)
if not os.path.exists(text_path):
    with open(text_path, 'w') as text_file:
        text_file.write(text_content)

metadata = {
    f"{data_dir}/{text_file_name}": {
        "title": text_file_name.split('.')[0],
        "file_path": text_path
    }
}

**Show text data**

In [6]:
def show_text(text_path):
    if os.path.isfile(text_path):
        with open(text_path, 'r') as text_file:
            contents = text_file.read()
        print(contents[:500])
        print("="*80)

In [7]:
show_text(text_path)

Madam Speaker, Madam Vice President, our First Lady and Second Gentleman. Members of Congress and the Cabinet. Justices of the Supreme Court. My fellow Americans.  

Last year COVID-19 kept us apart. This year we are finally together again. 

Tonight, we meet as Democrats Republicans and Independents. But most importantly as Americans. 

With a duty to one another to the American people to the Constitution. 

And with an unwavering resolve that freedom will always triumph over tyranny. 

Six day


## KDB.ai Vector Database - session and tables

**Libraries**

In [8]:
import kdbai_client as kdbai
from getpass import getpass

**KDB.ai session**

With the embeddings created, we need to store them in a vector database to enable efficient searching.

### Define KDB.AI Session

KDB.AI comes in two offerings:

1. [KDB.AI Cloud](https://trykdb.kx.com/kdbai/signup/) - For experimenting with smaller generative AI projects with a vector database in our cloud.
2. [KDB.AI Server](https://trykdb.kx.com/kdbaiserver/signup/) - For evaluating large scale generative AI applications on-premises or on your own cloud provider.

Depending on which you use there will be different setup steps and connection details required.

##### Option 1. KDB.AI Cloud

To use KDB.AI Cloud, you will need two session details - a URL endpoint and an API key.
To get these you can sign up for free [here](https://trykdb.kx.com/kdbai/signup).

You can connect to a KDB.AI Cloud session using `kdbai.Session` and passing the session URL endpoint and API key details from your KDB.AI Cloud portal.

If the environment variables `KDBAI_ENDPOINTS` and `KDBAI_API_KEY` exist on your system containing your KDB.AI Cloud portal details, these variables will automatically be used to connect.
If these do not exist, it will prompt you to enter your KDB.AI Cloud portal session URL endpoint and API key details.

In [9]:
KDBAI_ENDPOINT = (
    os.environ["KDBAI_ENDPOINT"]
    if "KDBAI_ENDPOINT" in os.environ
    else input("KDB.AI endpoint: ")
)
KDBAI_API_KEY = (
    os.environ["KDBAI_API_KEY"]
    if "KDBAI_API_KEY" in os.environ
    else getpass("KDB.AI API key: ")
)

session = kdbai.Session(api_key=KDBAI_API_KEY, endpoint=KDBAI_ENDPOINT)

##### Option 2. KDB.AI Server

To use KDB.AI Server, you will need download and run your own container.
To do this, you will first need to sign up for free [here](https://trykdb.kx.com/kdbaiserver/signup/).

You willreceive an email with the required license file and bearer  token needed to download your instance.
Follow instructions in the signup email to get your session up and running.

Once the [setup steps](https://code.kx.com/kdbai/gettingStarted/kdb-ai-server-setup.html) are complete you can then connect to your KDB.AI Server session using `kdbai.Session` and passing your local endpoint.

In [ ]:
# session = kdbai.Session(endpoint="http://localhost:8082")

**KDB.AI table**

In [11]:
# Table - name & schema
table_name = "sqqe_docs"

table_schema = [
        dict(name="document_id", type="bytes"),
        dict(name="text", type="bytes"),
        dict(name="embeddings", type="float32s"),
        dict(name="title", type="str"),
        dict(name="file_path", type="str")
    ]

indexFlat = {
        "name": "flat",
        "type": "flat",
        "column": "embeddings",
        "params": {'dims': 768, 'metric': 'L2'},
    }

In [12]:
# Connect with kdbai database
db = session.database("default")

In [13]:
# Drop table if exists
try:
    db.table(table_name).drop()
except kdbai.KDBAIException:
    pass

In [14]:
# Create table
table = db.create_table(table_name, table_schema, indexes=[indexFlat])

## Loading data

**Libraries**

In [15]:
from llama_index.core import SimpleDirectoryReader

**Loading data with metadata**

In [16]:
# Helper function - for getting metadata
def get_metadata(file_path):
    return metadata[file_path]

In [17]:
%%time

local_files = [fpath for fpath in metadata]
documents = SimpleDirectoryReader(input_files=local_files, file_metadata=get_metadata)

docs = documents.load_data()
len(docs)

CPU times: user 11.3 ms, sys: 0 ns, total: 11.3 ms
Wall time: 10.3 ms


1

## Creating Vector Store Index for data

**OpenAI API Key**

In [18]:
from getpass import getpass

In [19]:
os.environ["OPENAI_API_KEY"] = (
    os.environ["OPENAI_API_KEY"]
    if "OPENAI_API_KEY" in os.environ
    else getpass("OpenAI API key: ")
)


**Text embeddings model**

In [ ]:
from llama_index.embeddings.huggingface import HuggingFaceEmbedding

In [ ]:
EMBEDDING = "sentence-transformers/all-mpnet-base-v2"
embeddings_model = HuggingFaceEmbedding(model_name=EMBEDDING)

**LLM model**

In [22]:
from llama_index.llms.openai import OpenAI

In [23]:
LLM = "gpt-4o-mini"
llm_model = OpenAI(temperature=0, model=LLM)

**Setting callbacks and debug handler**

In [24]:
from llama_index.core.callbacks import LlamaDebugHandler
from llama_index.core.callbacks import CallbackManager

In [25]:
# Using the LlamaDebugHandler to print the trace of the sub questions captured by the SUB_QUESTION callback event type
llama_debug = LlamaDebugHandler(print_trace_on_end=True)
callback_manager = CallbackManager([llama_debug])

**Create vector store, storage context and the index for retrieval, query purposes**

In [26]:
from llama_index.vector_stores.kdbai import KDBAIVectorStore
from llama_index.core import StorageContext
from llama_index.core import Settings
from llama_index.core.indices import VectorStoreIndex
from llama_index.core.node_parser import SentenceSplitter

In [27]:
%%time

# Vector Store
text_store = KDBAIVectorStore(table=table)

# Storage context
storage_context = StorageContext.from_defaults(vector_store=text_store)

# Settings
Settings.callback_manager = callback_manager
Settings.transformations = [SentenceSplitter(chunk_size=500, chunk_overlap=0)]
Settings.embed_model = embeddings_model
Settings.llm = llm_model

# Vector Store Index
index = VectorStoreIndex.from_documents(
    docs,
    use_async=True,
    storage_context=storage_context,
)

**********
Trace: index_construction
    |_embedding -> 9.827609 seconds
    |_embedding -> 9.82735 seconds
**********
CPU times: user 7.24 s, sys: 4.45 s, total: 11.7 s
Wall time: 10.5 s


## Setup sub question query engine

**Index as Vector Query Engine**

In [28]:
# Vector query engine
vector_query_engine = index.as_query_engine(
                                vector_store_kwargs={
                                    "index" : "flat",
                                },
                            )

**Setting up Sub Question Query Engine**

In [29]:
from llama_index.core.tools import QueryEngineTool, ToolMetadata
from llama_index.core.query_engine import SubQuestionQueryEngine

In [30]:
# setup base query engine as tool
query_engine_tools = [
    QueryEngineTool(
        query_engine=vector_query_engine,
        metadata=ToolMetadata(
            name="state_of_union",
            description="State of Union Speech",
        ),
    ),
]

query_engine = SubQuestionQueryEngine.from_defaults(
    query_engine_tools=query_engine_tools,
    use_async=True,
)

**Querying the Sub Question Query Engine**

In [31]:
response = query_engine.query(
    "what did the president say about ukraine, what are the four common sense steps and how he planned to fight inflation?"
)

Generated 3 sub questions.
[state_of_union] Q: What did the president say about Ukraine in the State of the Union speech?
[state_of_union] Q: What are the four common sense steps mentioned by the president in the State of the Union speech?
[state_of_union] Q: How does the president plan to fight inflation according to the State of the Union speech?
[state_of_union] A: The four common sense steps mentioned by the president in the State of the Union speech are to stay protected with vaccines and treatments, ensure vaccination and boosting for the highest degree of protection, continue efforts to vaccinate more Americans, and maintain vigilance against the virus as it mutates and spreads.
[state_of_union] A: The president expressed strong support for Ukraine, highlighting the courage and determination of the Ukrainian people in their defense against Russian aggression. He emphasized that the United States stands with Ukraine and is providing over $1 billion in direct assistance to help th

In [32]:
print(response)

The president expressed strong support for Ukraine, highlighting the courage and determination of the Ukrainian people in their defense against Russian aggression. He affirmed that the United States stands with Ukraine, providing over $1 billion in direct assistance, while clarifying that U.S. forces would not engage in conflict with Russian forces in Ukraine. He acknowledged the challenges Ukraine would face and emphasized the coordinated international response to support them.

The four common sense steps mentioned are to stay protected with vaccines and treatments, ensure vaccination and boosting for the highest degree of protection, continue efforts to vaccinate more Americans, and maintain vigilance against the virus as it mutates and spreads.

To fight inflation, the president plans to lower costs rather than wages, increase the production of goods in America, and enhance infrastructure and innovation. Key components include capping the cost of prescription drugs, closing tax loo

**Iterate through all subquestions captured in SUB_QUESTION event**

In [33]:
from llama_index.core.callbacks import CBEventType, EventPayload

In [34]:
for i, (start_event, end_event) in enumerate(
    llama_debug.get_event_pairs(CBEventType.SUB_QUESTION)
):
    end_event_exception = end_event.payload.get(EventPayload.EXCEPTION)
    if end_event_exception is None:
        qa_pair = end_event.payload[EventPayload.SUB_QUESTION]
        print("Sub Question " + str(i) + ": " + qa_pair.sub_q.sub_question.strip())
        print("Answer: " + qa_pair.answer.strip())
        print("="*80)

Sub Question 0: What did the president say about Ukraine in the State of the Union speech?
Answer: The president expressed strong support for Ukraine, highlighting the courage and determination of the Ukrainian people in their defense against Russian aggression. He emphasized that the United States stands with Ukraine and is providing over $1 billion in direct assistance to help them. While clarifying that U.S. forces would not engage in conflict with Russian forces in Ukraine, he stated that American military resources are mobilized to defend NATO allies. The president also acknowledged the challenges Ukraine would face in the coming days and weeks but affirmed that the Ukrainian people would not tolerate attempts to undermine their independence. He mentioned the coordinated international response to support Ukraine and the economic measures being taken against Russia.
Sub Question 1: What are the four common sense steps mentioned by the president in the State of the Union speech?
Ans

## Delete the KDB.AI Table
Once finished with the table, it is best practice to drop it.

In [35]:
table.drop()